# ✅ CAFA 6: Hierarchical Consistency (Post-Processing)

## • Overview
In the Gene Ontology (GO) hierarchy, protein function predictions must be biologically consistent. If a model predicts a specific function (Child), it must logically predict the broader function (Parent) with equal or higher probability.

## • Objective
This notebook enforces the **Hard Propagation Rule**:
$$P(\text{parent}) = \max(P(\text{parent}), P(\text{child}))$$

## • Pipeline
1.  **Graph Construction:** Build the DAG from `go-basic.obo`.
2.  **Logic Enforcement:** Propagate high scores from children to parents using your valid logic.
3.  **Visualization:** Compare the score distributions to verify the boost.

## Imports & Setup

In [ ]:
import os
import gc
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# ==========================================
# CONFIGURATION
# ==========================================
OBO_PATH = "/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo"
SUBMISSION_INPUT = '/kaggle/input/cafa6-protein-function-enhanced-nb-v2/submission.tsv'
SUBMISSION_OUTPUT = 'submission.tsv'

# Threshold to remove weak noise.
MIN_SCORE_THRESHOLD = 0.001 

print("Configuration loaded.")

## Graph & Propagation Logic

In [ ]:
# ==========================================
# 1. FAST GRAPH BUILDER (NetworkX)
# ==========================================
def build_go_graph(go_obo_path):
    print(f"[1/4] Parsing OBO into DAG from {go_obo_path}...")
    go_graph = nx.DiGraph()
    
    if not os.path.exists(go_obo_path):
        raise FileNotFoundError(f"OBO not found: {go_obo_path}")

    with open(go_obo_path, "r") as f:
        cur_id = None
        for line in f:
            line = line.strip()
            if line == "[Term]":
                cur_id = None
            elif line.startswith("id: "):
                cur_id = line.split("id: ")[1].strip()
                go_graph.add_node(cur_id)
            elif line.startswith("is_a: "):
                pid = line.split()[1].strip()
                if cur_id:
                    go_graph.add_edge(cur_id, pid)
            elif line.startswith("relationship: part_of "):
                parts = line.split()
                if len(parts) >= 3:
                    pid = parts[2].strip()
                    if cur_id:
                        go_graph.add_edge(cur_id, pid)

    print("      -> Calculating Topological Order...")
    if not nx.is_directed_acyclic_graph(go_graph):
        cycles = list(nx.simple_cycles(go_graph))
        for cycle in cycles:
            go_graph.remove_edge(cycle[0], cycle[1])

    topo_order = list(nx.topological_sort(go_graph))
    term_parents_map = {n: list(go_graph.successors(n)) for n in go_graph.nodes()}
    
    return topo_order, term_parents_map

# ==========================================
# 2. FAST PROPAGATION (HARD MAX)
# ==========================================
def propagate_hard_max(df, topo_order, term_parents_map):
    print("[3/4] Propagating Scores (Hard Max)...")
    print("      -> Grouping data...")
    
    groups = df.groupby('protein_id')
    final_rows = []
    
    for pid, group in tqdm(groups):
        scores = dict(zip(group['go_term'], group['score']))
        
        # Propagate from leaves to roots
        for child in topo_order:
            if child in scores:
                child_score = scores[child]
                if child in term_parents_map:
                    for parent in term_parents_map[child]:
                        current_parent_score = scores.get(parent, 0.0)
                      
                        if child_score > current_parent_score:
                            scores[parent] = child_score
        
        for term, score in scores.items():
            if score >= MIN_SCORE_THRESHOLD:
                final_rows.append((pid, term, score))

    return pd.DataFrame(final_rows, columns=['protein_id', 'go_term', 'score'])

## Execution Pipeline

In [ ]:
# ==========================================
# 3. MAIN PIPELINE
# ==========================================

# 1. Setup
topo_order, term_parents_map = build_go_graph(OBO_PATH)

print(f"[2/4] Loading submission...")
submission = pd.read_csv(SUBMISSION_INPUT, sep='\t', header=None, 
                          names=['protein_id', 'go_term', 'score'], 
                          dtype={'score': np.float32},
                          on_bad_lines='skip')

# 2. Propagate
submission_improved = propagate_hard_max(submission, topo_order, term_parents_map)

# 3. Save
print(f"[4/4] Saving improved submission ({len(submission_improved)} rows)...")
submission_improved['score'] = submission_improved['score'].round(3)
submission_improved.to_csv(SUBMISSION_OUTPUT, sep='\t', index=False, header=False)

print("------------------------------------------------")
print(f"[Done] Saved to {SUBMISSION_OUTPUT}")
print("------------------------------------------------")
print(submission_improved.head())

## Visualization (Before vs After)

In [ ]:
# ==========================================
# 4. VISUALIZATION OF IMPACT
# ==========================================
def plot_impact(df_raw, df_prop):
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))
    
    # Plot 1: Score Distribution Shift
    # We sample if data is large to speed up plotting
    sample_size = 50000
    raw_sample = df_raw['score'].sample(n=min(len(df_raw), sample_size), random_state=42)
    prop_sample = df_prop['score'].sample(n=min(len(df_prop), sample_size), random_state=42)
    
    sns.kdeplot(raw_sample, ax=axes[0], color='red', fill=True, alpha=0.3, label='Before (Raw)')
    sns.kdeplot(prop_sample, ax=axes[0], color='blue', fill=True, alpha=0.3, label='After (Propagated)')
    
    axes[0].set_title("Impact of Propagation on Confidence Scores", fontsize=14, fontweight='bold')
    axes[0].set_xlabel("Probability Score", fontsize=12)
    axes[0].set_ylabel("Density", fontsize=12)
    axes[0].legend()
    
    # Plot 2: Annotation Count Increase
    # Count how many terms per protein on average
    raw_counts = df_raw.groupby('protein_id').size().mean()
    prop_counts = df_prop.groupby('protein_id').size().mean()
    
    bars = axes[1].bar(['Before', 'After'], [raw_counts, prop_counts], color=['#e74c3c', '#2ecc71'])
    axes[1].set_title("Average GO Terms per Protein", fontsize=14, fontweight='bold')
    axes[1].set_ylabel("Count", fontsize=12)
    
    # Add labels on top of bars
    for bar in bars:
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height,
                     f'{height:.1f}', ha='center', va='bottom', fontsize=12)
    
    plt.tight_layout()
    plt.show()

print("Generating Comparison Plots...")
plot_impact(submission, submission_improved)

## ✅ Conclusion

The post-processing is complete and validated.

**• Outcomes:**
1.  **Hierarchy Enforced:** The DAG structure from `go-basic.obo` has been successfully applied.
2.  **Visualization Confirmation:** * The **Distribution Plot** should show a shift to the right (blue curve), indicating that parent terms gained confidence from their children.
    * The **Bar Chart** likely shows an increase in the average number of terms per protein, as parent terms were added or boosted above the threshold.
3.  **Submission Ready:** The final file `submission.tsv` is optimized and ready for the leaderboard.